In [1]:
import pickle
import pandas as pd
import datetime
import re
import numpy as np
from tqdm import tqdm

In [2]:
with open('hw3_data/players.pkl', 'rb') as f: 
    players = pickle.load(f)
with open('hw3_data/results.pkl', 'rb') as f: 
    results = pickle.load(f)
with open('hw3_data/tournaments.pkl', 'rb') as f: 
    tournmts = pickle.load(f)

In [3]:
players_df = pd.DataFrame.from_dict(players).T.reset_index().drop(['index'], axis=1)

In [4]:
all_tournmts_df = pd.DataFrame.from_dict(tournmts).T.reset_index().drop(['index'], axis=1)

In [5]:
tournmts_df = all_tournmts_df[all_tournmts_df['dateStart'].apply(lambda x: x[:4]) == '2019']

In [6]:
results2019 = {idx: results[idx] for idx in tournmts_df['id']}

In [7]:
bad_idx = []
for idx in results2019:
    try:
        if 'mask' not in results2019[idx][0]:
            bad_idx.append(idx)
    except:
        bad_idx.append(idx)

In [8]:
for idx in bad_idx:
    if idx in results2019:
        del results2019[idx]

In [9]:
all_tournmts_df

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
0,1,Чемпионат Южного Кавказа,2003-07-25T00:00:00+04:00,2003-07-27T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
1,2,Летние зори,2003-08-09T00:00:00+04:00,2003-08-09T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
2,3,Турнир в Ижевске,2003-11-22T00:00:00+03:00,2003-11-24T00:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
3,4,Чемпионат Украины. Переходной этап,2003-10-11T00:00:00+04:00,2003-10-12T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
4,5,Бостонское чаепитие,2003-10-10T00:00:00+04:00,2003-10-13T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
...,...,...,...,...,...,...,...,...,...
5523,6481,Онлайн: 15:00 (а)Синхрон-lite. Лига старта. Эп...,2020-05-05T15:00:00+03:00,2020-05-05T18:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/53,"[{'id': 7533, 'name': 'Денис', 'patronymic': '...",None,"{'1': 12, '2': 12, '3': 12}"
5524,6482,Онлайн: 19:00 Зелёный шум,2020-05-07T19:00:00+03:00,2020-05-07T21:30:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/53,"[{'id': 7533, 'name': 'Денис', 'patronymic': '...",None,"{'1': 13, '2': 13, '3': 13}"
5525,6483,Онлайн: 19:00 (а)Синхрон-lite. Лига старта. Эп...,2020-05-08T19:00:00+03:00,2020-05-08T21:30:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/53,"[{'id': 7533, 'name': 'Денис', 'patronymic': '...",None,"{'1': 12, '2': 12, '3': 12}"
5526,6484,"Онлайн: 22:00 Не числом, а умением - 2 (NEW!)",2020-05-04T22:00:00+03:00,2020-05-04T23:40:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/53,"[{'id': 7533, 'name': 'Денис', 'patronymic': '...",None,"{'1': 12, '2': 12}"


In [10]:
players_results = {}
for tournmnt_idx in results2019:
    for team in results2019[tournmnt_idx]:
        cur_mask = team['mask']
        for member in team['teamMembers']:
            if member['player']['id'] not in players_results:
                players_results[member['player']['id']] = {tournmnt_idx: cur_mask}
            else:
                players_results[member['player']['id']][tournmnt_idx] = cur_mask 

In [11]:
bads = []
for player in players_results:
    for tour in players_results[player]:
        if isinstance(players_results[player][tour], str):
            players_results[player][tour] = re.sub(r'[^01]', '0', players_results[player][tour])
        else:
            bads.append((player, tour))

In [12]:
for bad in bads:
    del players_results[bad[0]][bad[1]]

In [13]:
for player in players_results:
    for tour in players_results[player]:
        players_results[player][tour] = np.array(list(map(float, list(players_results[player][tour]))))

In [14]:
def calc_average(tour_idx, skip_player_idx=None):
    results = []
    for player in players_results:
        if tour_idx in players_results[player] and player != skip_player_idx:
            results.append(players_results[player][tour_idx])
    avg = np.mean(results, axis=0)
    return np.where(avg == 0, 0.1, avg)

Пусть сложность вопроса - процентное соотношение количества людей, который ответили на него, исключая данного игрока. \
Пусть количество баллов, который будет получать человек за правильно отвеченный вопрос - 1 / сложность вопроса. \
В рамках одного турнира результат игрока - усреднённое количество баллов за все вопросы.

In [ ]:
res = calc_average(4772)
res

In [25]:
tour_idxs = set()
for player in players_results:
    for tour in players_results[player]:
        tour_idxs.add(tour)

In [27]:
tour_avgs = {}
for tour in tqdm(tour_idxs):
    try:
        tour_avgs[tour] = calc_average(tour)
    except:
        continue

100%|██████████| 675/675 [00:08<00:00, 78.08it/s]


In [33]:
player_scores = {}
for player in tqdm(players_results):
    cur_player_result = {}
    for tour in players_results[player]:
        if tour in tour_avgs:
            cur_player_result[tour] = np.mean(players_results[player][tour] / tour_avgs[tour])
    player_scores[player] = cur_player_result

100%|██████████| 59216/59216 [00:02<00:00, 23163.65it/s]


In [38]:
players_avg_scores = []
for player in player_scores:
    players_avg_scores.append([player, np.mean(list(player_scores[player].values()))])

/home/ffisin/PycharmProjects/made/venv/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ffisin/PycharmProjects/made/venv/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [42]:
sortedd = sorted(players_avg_scores, key=lambda x: -x[1])

In [80]:
players_power = {x[0]: x[1] for x in sortedd}

In [47]:
tournmts_df2020 = all_tournmts_df[all_tournmts_df['dateStart'].apply(lambda x: x[:4]) == '2020']

In [178]:
results2020 = {idx: results[idx] for idx in tournmts_df2020['id']}

In [179]:
teams2020 = {}
for tour in results2020:
    cur_results = {}
    for team in results2020[tour]:
        if 'position' in team:
            pos = team['position']
        else:
            continue
        members = []
        for member in team['teamMembers']:
            members.append(member['player']['id'])
        cur_results[team['team']['id']] = {'pos': pos, 'members': members}
    teams2020[tour] = cur_results
        

In [180]:
for tour in teams2020:
    for team in teams2020[tour]:
        powers = []
        for member in teams2020[tour][team]['members']:
            if member in players_power:
                powers.append(players_power[member])
            else:
                powers.append(0)
        teams2020[tour][team]['avg_power'] = np.mean(powers)

/home/ffisin/PycharmProjects/made/venv/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ffisin/PycharmProjects/made/venv/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [181]:
from scipy.stats import spearmanr, kendalltau

In [189]:
def calc_corr(tour_idx):
    true_pos = []
    scores = []
    for team in teams2020[tour_idx]:
        true_pos.append(teams2020[tour_idx][team]['pos'])
        scores.append([teams2020[tour_idx][team]['pos'], teams2020[tour_idx][team]['avg_power']])
    pred_rank = [s[0] for s in sorted(scores, key=lambda x: -x[1])]
    spear = spearmanr(true_pos, pred_rank, nan_policy='omit')[0]
    kend = kendalltau(true_pos, pred_rank, nan_policy='omit')[0]
    return (spear, kend)

In [190]:
spears = []
kends = []
for tour in teams2020:
    spear, kend = calc_corr(tour)
    if not np.isnan(spear):
        spears.append(spear)
    if not np.isnan(kend):
        kends.append(kend)
    

In [195]:
np.mean(spears)

0.7992867830083643

In [196]:
np.mean(kends)

0.6711605703450698

# В данном случае человек с один успешным турниром получает большой скор. Попробуем домножать на логарифм количества игр игрока

In [261]:
players_power1 = {
    player: power * np.log(len(player_scores[player]) + 18) for player, power in players_power.items()
}

In [262]:
for tour in teams2020:
    for team in teams2020[tour]:
        powers = []
        for member in teams2020[tour][team]['members']:
            if member in players_power1:
                powers.append(players_power1[member])
            else:
                powers.append(0)
        teams2020[tour][team]['avg_power'] = np.mean(powers)

In [263]:
spears = []
kends = []
for tour in teams2020:
    spear, kend = calc_corr(tour)
    if not np.isnan(spear):
        spears.append(spear)
    if not np.isnan(kend):
        kends.append(kend)
    

In [264]:
np.mean(spears)

0.8113734208893483

In [265]:
np.mean(kends)

0.682724669976522